In [1]:
import pandas as pd
import numpy as np
import rasterio
import time
import random

In [2]:
#df_insert = pd.DataFrame({'tmin_2015': [np.NaN]*7,
                    #'tmax_2015': [np.NaN]*7})
def dataframe_extraction(r1,row,col):
    """This function returns a pandas dataframe that holds the gdd values for 1-365 doys
    inputs: r1: the gdd RasterStack, which has 365 layers one corresponding to a doy of that year.
            row, col: the row and column of the raster pixel we would like to get the dataframe for """
    id=np.array(range(1,1+r1.shape[0]))
    data_frame = (pd.DataFrame({'DOY':range(1,366),
                                'gdd':r1[0:1+r1.shape[0]:1,row,col]},index=id))
    #print(data_frame)
    #df=pd.DataFrame()
    #for i in range(len(data_frame)):
    #    df=pd.concat([df,data_frame.iloc[[i]],df_insert])
    data_frame.index=range(1,1+len(data_frame.index))
    return data_frame

In [20]:
from multiprocessing import Pool
def initialize_rasters(path1,path2,path3):
    raster1=rasterio.open(path1)
    tot_cols=raster1.width
    tot_rows=raster1.height
    a=raster1.read()
    print("Tot rows: ",tot_rows," Tot cols: ",tot_cols)
    raster2=rasterio.open(path2)
    b=raster2.read()
    raster3=rasterio.open(path3)
    c=raster3.read()
    return (a[0,:,:],b[0,:,:],c,tot_rows,tot_cols)
def gdd_backcalculation(index,info,gdd_corn_green=340,gdd_soy_green=400,gdd_corn_sen=150,gdd_soy_sen=260):
    
    crop=info[0]
    green=info[1]
    sen=info[2]
    gdd=info[3]
    row=int(index/tot_cols)
    col=index-(tot_cols*row)
    g=green[row][col]
    s=sen[row][col]
    temp_data=dataframe_extraction(gdd,row,col)
    if crop=='corn':
        t=temp_data.loc[(temp_data.DOY<=g)]
        t.index=range(1,1+len(t))
        t['gdd_corn_cum']=t.loc[::-1, 'gdd'].cumsum()[::-1]
        #t['gdd_soy_cum']=t.loc[::-1, 'gdd'].cumsum()[::-1]
        c_cum=t['gdd_corn_cum']
        #s_cum=t['gdd_soy_cum']
        corn=c_cum[c_cum<gdd_corn_green]
        #soy=s_cum[s_cum<gdd_soy_green]
        p_corn=(corn.index[0]-1 if(corn.index[0]>1) else 100)
        #p_soy=(soy.index[0]-1 if(soy.index[0]>1) else 100)

        t=temp_data.loc[(temp_data.DOY>=s)]
        t.index=range(s,s+len(t))
        t['gdd_corn_cum']=t.loc[:,'gdd'].cumsum()
        #t['gdd_soy_cum']=t.loc[:,'gdd'].cumsum()
        c_cum=t['gdd_corn_cum']
        #s_cum=t['gdd_soy_cum']
        corn=c_cum[c_cum>gdd_corn_sen]
        #soy=s_cum[s_cum>gdd_soy_sen]
        h_corn=(corn.index[0] if len(corn.index>0) else 320)
        #h_soy=(soy.index[0] if len(soy.index>0) else 320)
        print((p_corn,h_corn))
        return (p_corn,h_corn)
    elif crop=='soy':
        t=temp_data.loc[(temp_data.DOY<=g)]
        t.index=range(1,1+len(t))
        #t['gdd_corn_cum']=t.loc[::-1, 'gdd'].cumsum()[::-1]
        t['gdd_soy_cum']=t.loc[::-1, 'gdd'].cumsum()[::-1]
        #c_cum=t['gdd_corn_cum']
        s_cum=t['gdd_soy_cum']
        #corn=c_cum[c_cum<gdd_corn_green]
        soy=s_cum[s_cum<gdd_soy_green]
        #p_corn=(corn.index[0]-1 if(corn.index[0]>1) else 100)
        p_soy=(soy.index[0]-1 if(soy.index[0]>1) else 100)

        t=temp_data.loc[(temp_data.DOY>=s)]
        t.index=range(s,s+len(t))
        #t['gdd_corn_cum']=t.loc[:,'gdd'].cumsum()
        t['gdd_soy_cum']=t.loc[:,'gdd'].cumsum()
        #cum=t['gdd_corn_cum']
        s_cum=t['gdd_soy_cum']
        #corn=c_cum[c_cum>gdd_corn_sen]
        soy=s_cum[s_cum>gdd_soy_sen]
        #h_corn=(corn.index[0] if len(corn.index>0) else 320)
        h_soy=(soy.index[0] if len(soy.index>0) else 320)
        return (p_soy,h_soy)
    #return (p_corn,p_soy,h_corn,h_soy)

In [11]:
src=rasterio.open('greenup_corn.tif')
crop='corn'
a,b,c,tot_rows,tot_cols=initialize_rasters('greenup_corn.tif','sen_corn.tif','gdd_corn_2012.tif')
y=[(crop,a,b,c)]*tot_rows*tot_cols
ind=range(tot_rows*tot_cols)
l=list()
l=list(list(zip(ind,y))[:])
plant=a*0
har=a*0
plant=plant.astype('int32')
har=har.astype('int32')

Tot rows:  757  Tot cols:  1521


In [19]:
len(l[575698][1])

4

In [21]:
start=time.time()
with Pool(processes=3) as pool:
    ind_start=575698+500
    ind_end=ind_start+10
    pairs=pool.starmap(gdd_backcalculation,l[ind_start:ind_end])
    pool.close()
    pool.join()
end=time.time()
print(end-start)
print(pairs)

/Users/koutilya/anaconda/envs/CCMS/lib/python3.5/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(135, 261)


/Users/koutilya/anaconda/envs/CCMS/lib/python3.5/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/koutilya/anaconda/envs/CCMS/lib/python3.5/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(127, 252)


/Users/koutilya/anaconda/envs/CCMS/lib/python3.5/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/koutilya/anaconda/envs/CCMS/lib/python3.5/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(127, 256)


/Users/koutilya/anaconda/envs/CCMS/lib/python3.5/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(125, 259)
(133, 259)
(132, 255)
(129, 259)
(129, 247)
(132, 268)
(132, 235)
121.43442988395691
[(135, 261), (127, 252), (127, 256), (125, 259), (133, 259), (132, 255), (129, 259), (129, 247), (132, 268), (132, 235)]


In [27]:
for index in list(range(ind_start,ind_end)):
    print(index)
    row=int(index/tot_cols)
    col=index-(tot_cols*row)
    plant[row][col]=pairs[index-ind_start][0]
    har[row][col]=pairs[index-ind_start][0]
profile=src.profile
profile.update(count=1)
print(profile)
with rasterio.open('plant.tif','w',**profile) as dst:
    dst.write(plant.astype(rasterio.uint16),1)
with rasterio.open('har.tif','w',**profile) as dst:
    dst.write(har.astype(rasterio.uint16),1)

576198
576199
576200
576201
576202
576203
576204
576205
576206
576207
{'interleave': 'band', 'transform': Affine(480.0, 0.0, -671205.0,
       0.0, -480.0, 2250435.0), 'count': 1, 'nodata': 0.0, 'driver': 'GTiff', 'height': 757, 'tiled': False, 'crs': CRS({'lon_0': -96, 'wktext': True, 'proj': 'aea', 'y_0': 0, 'ellps': 'GRS80', 'units': 'm', 'lat_1': 29.5, 'lat_2': 45.5, 'no_defs': True, 'towgs84': '0,0,0,0,0,0,0', 'lat_0': 23, 'x_0': 0}), 'dtype': 'uint16', 'width': 1521}


# The following is not required

In [4]:
def initialize_rasters(path1,path2):
    raster1=rasterio.open(path1)
    tot_cols=raster1.width
    tot_rows=raster1.height
    a=raster1.read()
    print("Tot rows: ",tot_rows," Tot cols: ",tot_cols)
    raster2=rasterio.open(path2)
    b=raster2.read()
    return (a,b,tot_rows,tot_cols)

a,b,tot_rows,tot_cols=initialize_rasters('PRISM_tmin_stable_4kmD1_2015_bil.tif','PRISM_tmax_stable_4kmD1_2015_bil.tif')

Tot rows:  621  Tot cols:  1405


621

In [35]:
y=[(a,b)]*tot_rows*tot_cols
ind=range(tot_rows*tot_cols)
l=list()
l=list(list(zip(ind,y))[:])

def gdd_calculation(index,rasters):
    a=rasters[0]
    b=rasters[1]
    tot_cols=a.shape[2]
    tot_rows=a.shape[1]
    row=int(index/tot_cols)
    col=index-(tot_cols*row)
    print("row: ",row," col: ",col)
    df=dataframe_extraction(a,b,row,col)
    df.clip(lower=pd.Series({'tmin_2015': 10.0, 'tmax_2015': 10.0}), upper=pd.Series({'tmax_2015': 25}), axis=1)
    df['tavg']=0.5*(df['tmax_2015']+df['tmin_2015'])
    df['gdd_soy']=df['tavg']-10
    return df

df=gdd_calculation(l[400000][0],l[400000][1])

872505


In [ ]:
print(df)